In [8]:
import pandas as pd

# Load the Excel file
df = pd.read_csv("d:\\Yehmh\\202309_大農大富RGB&NDVI\\DNDF_PlantLine2.csv")


# Function to calculate missing coordinates
def fill_coordinates(row):
    return row['latitude'] - 20 * 0.000008983 * row['block_y'], row['longitude'] - 20 * 0.000008983 * row['block_x']

# Apply the function to fill missing coordinates
df['latitude_0'], df['longitude_0'] = zip(*df.apply(fill_coordinates, axis=1))

# Display the updated dataframe
print(df)

# Save the updated dataframe to CSV
df.to_csv("d:\\Yehmh\\202309_大農大富RGB&NDVI\\DNDF_PlantLine_update.csv", index=False)

print("Updated coordinates saved to updated_coordinates.csv")

     Plot  line_num  block_x  block_y   latitude   longitude  Unnamed: 6  \
0      12         1        5        0  23.627039  121.425218         NaN   
1      12         1        5        1  23.627222  121.425200         NaN   
2      12         1        3        1  23.627200  121.424806         NaN   
3      12         1        0        1  23.627169  121.424224         NaN   
4      12         1        0        0  23.627000  121.424242         NaN   
..    ...       ...      ...      ...        ...         ...         ...   
131   146         2        1        5  23.618529  121.407121         NaN   
132   146         2        1        3  23.618186  121.407171         NaN   
133   146         2        1        0  23.617611  121.407206         NaN   
134   146         2        0        0  23.617627  121.407006         NaN   
135   146         2        0        3  23.617993  121.407007         NaN   

     latitude_0  longitude_0  
0     23.627039   121.424320  
1     23.627043   121.424

In [12]:
import pandas as pd

# Load the Excel file
df = pd.read_csv("d:\\Yehmh\\202309_大農大富RGB&NDVI\\DNDF_PlantLine2.csv")

# Custom function to filter rows
def filter_rows(group):
    if (group['block_x'] == 0).any() and (group['block_y'] == 0).any():
        return group[(group['block_x'] == 0) & (group['block_y'] == 0)]
    else:
        return group.iloc[[0]]

# Apply the function to groupby object and concatenate the results
filtered_df = df.groupby(['Plot', 'line_num']).apply(filter_rows).reset_index(drop=True)
print(filtered_df)

# Save the updated dataframe to CSV
filtered_df.to_csv("d:\\Yehmh\\202309_大農大富RGB&NDVI\\DNDF_PlantLine_00.csv", index=False)

print("Updated coordinates saved to updated_coordinates.csv")

    Plot  line_num  block_x  block_y   latitude   longitude  Unnamed: 6
0     12         1        0        0  23.627000  121.424242         NaN
1     14         1        0        0  23.624295  121.424852         NaN
2     16         1        0        0  23.621290  121.423978         NaN
3     26         1        0        0  23.626988  121.422896         NaN
4     29         1        0        0  23.622862  121.422567         NaN
5     31         1        0        0  23.619771  121.421850         NaN
6     41         1        0        0  23.625543  121.420197         NaN
7     46         1        0        0  23.618149  121.420583         NaN
8     65         1        0        0  23.631620  121.417112         NaN
9     68         1        0        0  23.626968  121.417072         NaN
10    70         1        0        0  23.624386  121.417296         NaN
11    79         1        0        0  23.631961  121.414875         NaN
12    81         1        0        0  23.628803  121.415585     

In [1]:
import csv
from twd97 import fromwgs84

# Function to read CSV file, convert coordinates, and write back to CSV
def convert_coordinates(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames + ['TWD97_X', 'TWD97_Y']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        
        for row in reader:
            latitude = float(row['latitude'])  # Assuming 'Latitude' is the column name for latitude
            longitude = float(row['longitude'])  # Assuming 'Longitude' is the column name for longitude
            
            # Convert WGS84 coordinates to TWD97
            twd97_x, twd97_y = fromwgs84(latitude, longitude)
            
            # Add TWD97 coordinates to the row
            row['TWD97_X'] = twd97_x
            row['TWD97_Y'] = twd97_y
            
            writer.writerow(row)

# Example usage
input_file = 'G:/Yehmh/DNDF/DNDF_PlantLine2.csv'
output_file = 'G:/Yehmh/DNDF/DNDF_PlantLine_twd97_all.csv'
convert_coordinates(input_file, output_file)


In [ ]:
import pandas as pd
import os

# Define the folder path containing the Excel files
folder_path = 'G:\\Yehmh\\DNDF\\tree_survey'

# Get a list of all Excel files in the folder
files = [file for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# Initialize an empty DataFrame to store the combined data
combined_data = pd.DataFrame()

# Loop through each Excel file and concatenate the relevant data
for file in files:
    # Read the Excel file into a DataFrame
    # print(file)
    df = pd.read_excel(os.path.join(folder_path, file))
    
    # Filter the relevant columns
    relevant_columns = ['Transect', 'Q20', 'Q10', 'Species', 'DBH']
    if 'Transect' not in df.columns and 'Plot' in df.columns:
        df.rename(columns={'Plot': 'Transect'}, inplace=True)

    df = df[relevant_columns]
    
    # Concatenate the data to the combined DataFrame
    combined_data = pd.concat([combined_data, df], ignore_index=True)

# Check for empty sequences and print them
empty_sequences = combined_data[combined_data['Species'].isnull()]
# print("Empty sequences:")
# print(empty_sequences)

# Drop the empty sequences before further processing if needed
combined_data = combined_data.dropna(subset=['Species'])

# Write the combined data to a new Excel file
print(combined_data)
combined_data.to_excel('G:\\Yehmh\\DNDF\\tree_survey_combined_2.xlsx', index=False)

print("Combined file has been created successfully!")

In [10]:
import pandas as pd
import twd97

# Read the CSV files into pandas DataFrames
file_path = "G:\\Yehmh\\DNDF\\tree_survey_combined.xlsx"
df = pd.read_excel(file_path)

# Group species data by trap location and find the most common species in each trap
main_species_df = df.groupby(['Plot', 'Transect', 'Q20', 'Q10'])['Species'].agg(lambda x: x.value_counts().idxmax()).reset_index()
# print(main_species_df)

# Calculate the counts of each species in each plot
species_counts = df.groupby(['Plot', 'Transect', 'Q20', 'Q10', 'Species']).size().reset_index(name='Count')
# print(species_counts)

# Calculate the total count of species in each plot
total_counts = species_counts.groupby(['Plot', 'Transect', 'Q20', 'Q10'])['Count'].sum().reset_index(name='TotalCount')
# print(total_counts)

# Merge total counts with species counts
species_counts = pd.merge(species_counts, total_counts, on=['Plot', 'Transect', 'Q20', 'Q10'], how='left')
# print(species_counts)

# # Calculate the percentage of the main species in each plot
species_counts['Percentage'] = (species_counts['Count'] / species_counts['TotalCount']) * 100
species_80up = species_counts[species_counts['Percentage'] >= 80]
print(species_80up)
# print(species_counts)

# # Now, you can save the merged DataFrame to a new CSV file if needed
# species_counts.to_csv("belt_species_percentage.csv", index=False)
# species_80up.to_excel("D:\\Yehmh\\202309_大農大富RGB&NDVI\\species_80up.xlsx", index = False)

      Plot  Transect Q20  Q10 Species  Count  TotalCount  Percentage
1       12         1   A  1.0     青剛櫟    120         129   93.023256
4       12         1   A  3.0     青剛櫟    134         134  100.000000
6       12         1   A  4.0     青剛櫟     50          52   96.153846
7       12         1   B  1.0     青剛櫟     88          88  100.000000
9       12         1   B  2.0     青剛櫟     66          68   97.058824
...    ...       ...  ..  ...     ...    ...         ...         ...
2370   156         1   D  4.0      杜英      3           3  100.000000
2371   156         1   E  1.0      杜英      8           8  100.000000
2372   156         1   E  2.0      杜英     10          12   83.333333
2374   156         1   E  3.0      杜英     13          14   92.857143
2376   156         1   E  4.0      杜英     14          15   93.333333

[471 rows x 8 columns]


In [24]:
import pandas as pd
import twd97

# Read the CSV files into pandas DataFrames
file_path = "G:\\Yehmh\\DNDF\\tree_survey_combined.xlsx"
df = pd.read_excel(file_path)

df['DBH_square'] = df['DBH'] ** 2
# print(df)

# DBH square count
species_dbh_square = df.groupby(['Plot', 'Transect', 'Q20', 'Q10', 'Species'])['DBH_square'].sum().reset_index(name='square_sum')
# print(dbh_square_count)

# Calculate the total DBH square count of species in each plot
total_dbh_square = species_dbh_square.groupby(['Plot', 'Transect', 'Q20', 'Q10'])['square_sum'].sum().reset_index(name='TotalSquareSum')

# Merge total counts with species counts
merge_df = pd.merge(species_dbh_square, total_dbh_square, on=['Plot', 'Transect', 'Q20', 'Q10'], how='left')
# print(merge_df)

# # Calculate the percentage of the main species in each plot
merge_df['Percentage'] = (merge_df['square_sum'] / merge_df['TotalSquareSum']) * 100
species_80up = merge_df[merge_df['Percentage'] >= 80]
print(species_80up)

# # Now, you can save the merged DataFrame to a new CSV file if needed
species_80up.to_excel("G:\\Yehmh\\DNDF\\species_80up_dbh.xlsx", index = False)

      Plot  Transect Q20  Q10 Species  square_sum  TotalSquareSum  Percentage
1       12         1   A  1.0     青剛櫟    10115.43        11530.23   87.729646
4       12         1   A  3.0     青剛櫟     8982.38         8982.38  100.000000
6       12         1   A  4.0     青剛櫟     6278.96         6495.28   96.669582
7       12         1   B  1.0     青剛櫟     3556.64         3556.64  100.000000
9       12         1   B  2.0     青剛櫟     4049.52         4392.74   92.186653
...    ...       ...  ..  ...     ...         ...             ...         ...
2370   156         1   D  4.0      杜英      857.78          857.78  100.000000
2371   156         1   E  1.0      杜英     2731.53         2731.53  100.000000
2372   156         1   E  2.0      杜英     3071.00         3372.97   91.047356
2374   156         1   E  3.0      杜英     3301.25         3326.25   99.248403
2376   156         1   E  4.0      杜英     2832.56         2967.12   95.464963

[588 rows x 8 columns]


In [34]:
import pandas as pd

def compare_excel_files(file1, file2):
    df1 = pd.read_excel(file1)
    df2 = pd.read_excel(file2)
    
    # Ensure both DataFrames have the same columns
    # if not all(df1.columns == df2.columns):
    #     raise ValueError("Columns in the two Excel files do not match.")
    
    # Select common columns for comparison
    common_columns = ['Plot', 'Transect', 'Q20', 'Q10']
    
    # Merge the two DataFrames on the common columns
    merged_df = pd.merge(df1, df2, on=common_columns, suffixes=('_file1', '_file2'))
    # print(merged_df)
    
    # Identify rows where species values are different
    diff_species = merged_df[merged_df['Species_file1'] != merged_df['Species_file2']]
    
    # Print out the differing rows
    if not diff_species.empty:
        print("Differences found in species value:")
        print(diff_species)

# Replace 'file1.xlsx' and 'file2.xlsx' with the paths to your Excel files
compare_excel_files('G:\\Yehmh\\DNDF\\species_80up.xlsx', 'G:\\Yehmh\\DNDF\\species_80up_dbh.xlsx')



Differences found in species value:
     Plot  Transect Q20  Q10 Species_file1  Count  TotalCount  \
147    65         1   E    4           杜虹花     18          22   
366   133         2   C    4           銀合歡      4           5   
383   146         1   A    3            陰香    140         166   
384   146         1   A    4            陰香     91         112   

     Percentage_file1 Species_file2  square_sum  TotalSquareSum  \
147         81.818182            茄苳    21250.00        22325.04   
366         80.000000            樟樹      645.16          717.66   
383         84.337349          台灣欒樹     4475.98         5168.68   
384         81.250000          台灣欒樹     4682.30         5220.68   

     Percentage_file2  
147         95.184600  
366         89.897723  
383         86.598126  
384         89.687550  


In [ ]:
import os
import shutil
import pandas as pd

def copy_tif_files(source_folder, destination_folder, min_x, max_x, min_y, max_y):
    # Iterate through files in the source folder
    for filename in os.listdir(source_folder):
        if filename.endswith(".tif"):
            # Remove the file extension
            filename_without_extension = os.path.splitext(filename)[0]
            # Assuming the filename follows a pattern where coordinates are included in the filename
            # Extracting coordinates from the filename
            # print(filename_without_extension)
            coordinates = filename_without_extension.split("_")
            x_coord = int(coordinates[0])
            y_coord = int(coordinates[1])
            # print(x_coord, y_coord)

            # Check if coordinates are within the specified range
            if min_x <= x_coord <= max_x and min_y <= y_coord <= max_y:
                # Copy the file to the destination folder
                shutil.copy(os.path.join(source_folder, filename), destination_folder)
                print(f"File {filename} copied.")

def Q10(Q10_value, min_x, min_y):
    if Q10_value == 1:
        return min_x, min_y
    if Q10_value == 2:
        return min_x, min_y+10
    if Q10_value == 3:
        return min_x+10, min_y+10
    if Q10_value == 4:
        return min_x+10, min_y

def find_Q(Q20_value, Q10_value, min_x, min_y):
    if Q20_value == 'A':
        min_x, min_y = Q10(Q10_value, min_x, min_y)
    if Q20_value == 'B':
        min_x, min_y = Q10(Q10_value, min_x+20, min_y)
    if Q20_value == 'C':
        min_x, min_y = Q10(Q10_value, min_x+40, min_y)
    if Q20_value == 'D':
        min_x, min_y = Q10(Q10_value, min_x+60, min_y)
    if Q20_value == 'E':
        min_x, min_y = Q10(Q10_value, min_x+80, min_y)
    
    return min_x, min_y

def find_ns_Q(Q20_value, Q10_value, min_x, min_y):
    if Q20_value == 'A':
        min_x, min_y = Q10(Q10_value, min_x, min_y)
    if Q20_value == 'B':
        min_x, min_y = Q10(Q10_value, min_x, min_y+20)
    if Q20_value == 'C':
        min_x, min_y = Q10(Q10_value, min_x, min_y+40)
    if Q20_value == 'D':
        min_x, min_y = Q10(Q10_value, min_x, min_y+60)
    if Q20_value == 'E':
        min_x, min_y = Q10(Q10_value, min_x, min_y+80)
    
    return min_x, min_y

# ------------------------------------------------------------------------- Example usage:
# Paths to your CSV files
coordination_csv_path = 'G:\\Yehmh\\DNDF\\DNDF_PlantLine_twd97.csv'
species_csv_path = 'G:\\Yehmh\\DNDF\\species_80up_dbh.xlsx'

# Read CSV files into pandas dataframes
coord_df = pd.read_csv(coordination_csv_path)
species_df = pd.read_excel(species_csv_path)

# Define folder paths
folder_paths = []
for i in range(1, 10):
    folder_paths.append(f"G:\\Yehmh\\DNDF\\202309_DNDF\\pix4d file\\00{i}\\5m_5m")
for i in range(10, 16):
    folder_paths.append(f"G:\\Yehmh\\DNDF\\202309_DNDF\\pix4d file\\0{i}\\5m_5m")

# Merge dataframes on common columns (plot and transect)
merged_df = pd.merge(coord_df, species_df, on=["Plot", "Transect"])
# print(merged_df)

# Iterate over each row in the species CSV file
for index, row in merged_df.iterrows():
    species = row["Species"]
    Plot = row["Plot"]
    Transect = row["Transect"]
    Q20_value = row["Q20"]
    Q10_value = row["Q10"]
    TWD97_X = row["TWD97_X"]
    TWD97_Y = row["TWD97_Y"]
    direction = row["direction"]

    print(f"Species: {species}, Plot: {Plot}, Transect: {Transect}, Q20: {Q20_value}, Q10: {Q10}")

    # Create folder if it doesn't exist
    species_folder = os.path.join('G:\\Yehmh\\DNDF\\202309_DNDF\\species_dbh', species)
    if not os.path.exists(species_folder):
        os.makedirs(species_folder)

    min_x = TWD97_X
    min_y = TWD97_Y

    if pd.isna(direction):
        min_x, min_y = find_Q(Q20_value, Q10_value, min_x, min_y)
    else:
        print(direction)
        min_x, min_y = find_ns_Q(Q20_value, Q10_value, min_x, min_y)

    # Iterate over each folder path
    for folder_path in folder_paths:
        copy_tif_files(folder_path, species_folder, min_x, min_x+10, min_y, min_y+10)

In [36]:
# rename file from chinese to english

import os
import pandas as pd

# Path to the Excel file containing original and new folder names
excel_file = 'G:\\Yehmh\\DNDF\\202309_DNDF\\plants_name.xlsx'
# Base path where your folders are located
base_path = 'G:\\Yehmh\\DNDF\\202309_DNDF\\species_dbh'

# Function to rename folders based on Excel mapping
def rename_folders_based_on_excel(excel_file, base_path):
    # Read the Excel file
    df = pd.read_excel(excel_file)
    # Get a list of original folder names from the Excel file
    original_names = df['original name'].tolist()
    # Get a list of new folder names from the Excel file
    new_names = df['new name'].tolist()
    
    # Iterate over folders in the base path
    for folder_name in os.listdir(base_path):
        # Check if the item is a directory
        folder_path = os.path.join(base_path, folder_name)
        if os.path.isdir(folder_path):
            # Check if the folder name is in the list of original names from the Excel file
            if folder_name in original_names:
                # If found, get the corresponding new name from the Excel file
                index = original_names.index(folder_name)
                new_name = new_names[index]
                new_folder_path = os.path.join(base_path, new_name)
                # Rename the folder
                os.rename(folder_path, new_folder_path)
                print(f'Renamed folder "{folder_path}" to "{new_folder_path}"')

# Call the function to rename folders based on Excel mapping
rename_folders_based_on_excel(excel_file, base_path)



Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\青剛櫟" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Quer_gl"
Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\茄苳" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Bisc_ja"
Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\白雞油" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Frax_gr"
Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\楓香" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Liqu_fo"
Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\台灣櫸" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Zelk_se"
Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\台灣欒樹" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Koel_el"
Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\烏心石" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Magn_co"
Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\杜英" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Elae_sy"
Renamed folder "G:\Yehmh\DNDF\202309_DNDF\species_dbh\印度紫檀" to "G:\Yehmh\DNDF\202309_DNDF\species_dbh\Pter_in"
Renamed fol